In [3]:
### LIBRERIAS

from dotenv import load_dotenv
from geopy.geocoders import Nominatim
import os 
import ssl
import certifi
import pandas as pd
import geopy.distance

# Deshabilitar la verificación SSL
ctx = ssl.create_default_context(cafile=certifi.where())
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

### 

In [4]:
load_dotenv()

True

In [5]:
API_KEY_GEOPY = os.getenv("API_KEY_GEOPY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [6]:
def coordenadas_direccion(direccion):

    global API_KEY_GEOPY

    # Inicializa el geocodificador
    geolocator = Nominatim(user_agent=API_KEY_GEOPY,ssl_context=ctx)

    # Geocodifica la dirección
    ubicacion = geolocator.geocode(direccion,addressdetails=True)

    # Validación 1: Verificar si la dirección se pudo geocodificar
    if not ubicacion:
        print("Error: No se pudo encontrar la dirección. Verifica si es correcta.")
        return None

    # Obtener la dirección devuelta y las coordenadas
    direccion_devuelta = ubicacion.address
    latitud = ubicacion.latitude
    longitud = ubicacion.longitude

    # Extraer la comuna, ciudad y país desde la información cruda
    detalle = ubicacion.raw['address']
    ciudad = detalle.get('city', detalle.get('town', detalle.get('village', 'No disponible')))
    pais = detalle.get('country', 'No disponible')

    # Validación 3: Verificar componentes clave como comuna, ciudad y país
    if ciudad == 'No disponible' or pais == 'No disponible':
        print("Error: La dirección no contiene información clave como la ciudad o el país.")
        return None

    # Si pasa todas las validaciones, la dirección es válida
    print("La dirección es válida.")
    print(f"Latitud: {latitud}, Longitud: {longitud}")
    print(f"Ciudad: {ciudad}, País: {pais}")

    # Devolver las coordenadas y la información relevante
    return {
        "direccion": direccion_devuelta,
        "Latitud": latitud,
        "Longitud": longitud,
        "ciudad": ciudad,
        "pais": pais
    }

In [8]:
direccion = 'Santiago, Región Metropolitana,Nataniel Cox 620'#"Juan Perez 123, Parral" # "Nataniel Cox 620, Santiago, Región Metropolitana"
resultado = coordenadas_direccion(direccion)

if resultado:
    print("Coordenadas y detalles de la dirección:")
    print(resultado)


La dirección es válida.
Latitud: -33.4535321, Longitud: -70.6523693
Ciudad: Santiago, País: Chile
Coordenadas y detalles de la dirección:
{'direccion': 'Banco Falabella, 620, Nataniel Cox, Barrio Copiapó, Santiago, Provincia de Santiago, Región Metropolitana de Santiago, 8330444, Chile', 'Latitud': -33.4535321, 'Longitud': -70.6523693, 'ciudad': 'Santiago', 'pais': 'Chile'}


In [86]:
dataset = pd.read_csv('Dataset_de_Locales.csv',sep=',')

In [83]:
dataset.head(3)

,fecha,local_id,local_nombre,comuna_nombre,localidad_nombre,local_direccion,funcionamiento_hora_apertura,funcionamiento_hora_cierre,local_telefono,local_lat,local_lng,funcionamiento_dia,fk_region,fk_comuna,fk_localidad
0,14-08-24,1,CRUZ VERDE,QUILLOTA,QUILLOTA,"OHIGGINS 195, LOCAL 1",08:30:00,18:30:00,56332269467,-32.879343,-71.246787,miercoles,6,69,32
1,14-08-24,3,CRUZ VERDE,LIMACHE,LIMACHE,URMENETA 99,08:30:00,18:30:00,56332415940,-32.984992,-71.275718,miercoles,6,59,17
2,14-08-24,4,CRUZ VERDE,LIMACHE,LIMACHE,REPUBLICA 281,08:30:00,18:30:00,56332418343,-33.002501,-71.265498,miercoles,6,59,17


In [84]:
# Función para calcular la distancia y filtrar el dataset para el local con la menor distancia

def farmacia_mas_cercana(referencia):
    
    # Leer el dataset y el archivo de referencia

    global dataset
    
    df = pd.DataFrame(dataset)
    punto_referencia = (referencia['Latitud'], referencia['Longitud'])

    # Calcular la distancia de cada local respecto al punto de referencia
    df['distancia'] = df.apply(
        lambda row: geopy.distance.geodesic((float(row['local_lat']), float(row['local_lng'])), punto_referencia).kilometers, axis=1
    )

    # Filtrar el dataset para el local con la menor distancia
    local_mas_cercano = df.loc[df['distancia'].idxmin()]

    return local_mas_cercano

In [88]:
farmacia_mas_cercana(resultado)

fecha                                14-08-24
local_id                                    4
local_nombre                       CRUZ VERDE
comuna_nombre                         LIMACHE
localidad_nombre                      LIMACHE
local_direccion                 REPUBLICA 281
funcionamiento_hora_apertura         08:30:00
funcionamiento_hora_cierre           18:30:00
local_telefono                    56332418343
local_lat                          -33.002501
local_lng                          -71.265498
funcionamiento_dia                  miercoles
fk_region                                   6
fk_comuna                                  59
fk_localidad                               17
distancia                           75.975135
Name: 2, dtype: object

In [89]:
def buscar_farmacia_mas_cercana(direccion):
    
    global dataset
    
    # Obtener las coordenadas de la dirección proporcionada
    referencia = coordenadas_direccion(direccion)
    if referencia is None:
        return None

    # Encontrar la farmacia más cercana
    local_mas_cercano = farmacia_mas_cercana(referencia)

    return local_mas_cercano

In [90]:
buscar_farmacia_mas_cercana(direccion)

La dirección es válida.
Latitud: -33.4539161, Longitud: -70.6524198
Ciudad: Santiago, País: Chile


fecha                                14-08-24
local_id                                    4
local_nombre                       CRUZ VERDE
comuna_nombre                         LIMACHE
localidad_nombre                      LIMACHE
local_direccion                 REPUBLICA 281
funcionamiento_hora_apertura         08:30:00
funcionamiento_hora_cierre           18:30:00
local_telefono                    56332418343
local_lat                          -33.002501
local_lng                          -71.265498
funcionamiento_dia                  miercoles
fk_region                                   6
fk_comuna                                  59
fk_localidad                               17
distancia                           75.975135
Name: 2, dtype: object

In [9]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import create_react_agent
from pprint import pprint

In [10]:
# Dirección como input externo
direccion_input = "Juan Perez 123, Parral"

In [11]:
model = ChatOpenAI(model="gpt-4o")

@tool
def buscar_farmacia_mas_cercana(direccion):
    
    """
    Encuentra la farmacia más cercana a la dirección proporcionada.

    Args:
        direccion (str): La dirección para la cual se quiere encontrar la farmacia más cercana.
        dataset (list): Un dataset que contiene las farmacias con sus coordenadas.

    Returns:
        dict: La farmacia más cercana con la información relevante.
    """
    # Obtener las coordenadas de la dirección proporcionada

    global dataset 

    referencia = coordenadas_direccion(direccion)
    if referencia is None:
        return None

    # Encontrar la farmacia más cercana
    local_mas_cercano = farmacia_mas_cercana(referencia)

    return local_mas_cercano


tools = [buscar_farmacia_mas_cercana]
system_message = "Eres un asistente que ayuda a encontrar farmacias cercanas basadas en la dirección proporcionada."

# Cambiar la consulta (query) usando la dirección como input
query = f"""
        Encuentra la farmacia más cercana a la siguiente dirección: {direccion_input}. 
        La respuesta debe venir estructurada, de la siguiente manera:

        - La Farmacia más cercana es: 
        - Su dirección es:
        - Su horario de Apertura es:
        - Su horario de Cierre es:
        - La distancia a su dirección de referencia es:

        Donde la distancia solo debe entregar dos decimales como máximo y debe venir sin estos strings "**".
        """

# Crear la aplicación React con LangChain
app = create_react_agent(model, tools, state_modifier=system_message)

# Invocar la aplicación con la consulta
messages = app.invoke({"messages": [("human", query)]})

Error: No se pudo encontrar la dirección. Verifica si es correcta.


In [105]:
from langchain.schema import AIMessage

In [136]:
ai_messages = [msg for msg in messages['messages'] if isinstance(msg, AIMessage)]

In [137]:
print(ai_messages[-1].content)

- La Farmacia más cercana es: CRUZ VERDE
- Su dirección es: REPUBLICA 281, LIMACHE
- Su horario de Apertura es: 08:30:00
- Su horario de Cierre es: 18:30:00
- La distancia a su dirección de referencia es: 75.98 km
